In [2]:
from selenium import webdriver
from time import sleep
import threading
import re
from tqdm import tnrange, tqdm_notebook

In [50]:
def get_item(page, driver): 
    
    # regex of item number
    regex = re.compile('[0-9]+\?page')
    
    driver.get("http://getitbeauty.m.tving.com/Product?page={}".format(page))
    sleep(0.5)

    lis = driver.find_elements_by_css_selector('.subType5 li dl') 

    
    idx = 0
    rows = []
    while(True):
        try: 
            li = lis[idx]
            
            img = li.find_element_by_css_selector('.thumb a img').get_attribute('src')
            name = li.find_element_by_css_selector('dt a').text

            brand = li.find_element_by_css_selector('dt a strong').text

            # split category
            cates = li.find_elements_by_css_selector('.gray a')
            cate_res = []
            for cate in cates:
                cate_res.append(cate.text)


            link = li.find_element_by_css_selector('.thumb a').get_attribute('href')

            # split item number
            matchobj = regex.search(link)
            item_num = matchobj.group()
            item_num = item_num.split('?')[0] 
             
            rows.append([item_num, img, name, brand, cate_res, link])
            
            idx = idx + 1
            if idx == len(lis):
                break
                
        except Exception as e:
            del rows[:]
            rows = [] 
            sleep(1) 
            
            lis = driver.find_elements_by_css_selector('.subType5 li dl') 
            idx = 0 
         
         
    return rows


def get_page(start, end):
    
    driver = webdriver.PhantomJS()
    for page in range(start, end):
        page_items = get_item(page, driver)
        all_items.extend(page_items)
        
    driver.close() 

In [51]:
!killall phantomjs

No matching processes belonging to you were found


431페이지마다 12개 아이템이 있으며 마지막 432페이즈는 8개만 있다.

In [78]:
(431 * 12) + 8

5180

스레드 6개로 분할하여 72페이지씩 크롤링 작업 수행

In [49]:
432 / 6

72.0

In [52]:
all_items = []
for cnt in range(1, 7): 
    start = 72 * (cnt-1)+1
    end = 72 * cnt+1 
    
    th = threading.Thread(target=get_page, args=(start, end))
    th.start()

In [77]:
len(all_items)

5180

In [83]:
item_df = pd.DataFrame(all_items, columns=['id_item','link','name','brand','category','img'])

In [89]:
item_df['cate1']= item_df['category'].apply(lambda x: x[0])
item_df['cate2']= item_df['category'].apply(lambda x: x[1])

In [92]:
item_df = item_df.drop(columns=['category'])

In [94]:
item_df.to_csv('data/item.csv', index=False)

In [307]:
item_df.head()

,id_item,link,name,brand,img,cate1,cate2
0,27111,http://img.lifestyler.co.kr/uploads/channel/pr...,[시세이도] 화이트 루센트 브라이트닝 스팟 컨트롤 파...,[시세이도],http://getitbeauty.m.tving.com/Product/Product...,MAKEUP,파운데이션
1,27110,http://img.lifestyler.co.kr/uploads/channel/pr...,[시세이도] 화이트 루센트 브라이트닝 스팟 컨트롤 베...,[시세이도],http://getitbeauty.m.tving.com/Product/Product...,MAKEUP,메이크업베이스
2,27109,http://img.lifestyler.co.kr/uploads/channel/pr...,[시세이도] 화이트 루센트 브라이트닝 프로텍티브 크림...,[시세이도],http://getitbeauty.m.tving.com/Product/Product...,SKINCARE,미백크림
3,27108,http://img.lifestyler.co.kr/uploads/channel/pr...,[시세이도] 화이트 루센트 인텐시브 스팟 타겟팅 세럼,[시세이도],http://getitbeauty.m.tving.com/Product/Product...,SKINCARE,화이트닝 에센스
4,27107,http://img.lifestyler.co.kr/uploads/channel/pr...,[시세이도] 화이트 루센트 인텐시브 브라이트닝 마스크,[시세이도],http://getitbeauty.m.tving.com/Product/Product...,SKINCARE,마스크팩


In [6]:
def get_item_detail(item, driver):
    driver.get("http://getitbeauty.m.tving.com/Product/ProductView/{}".format(item))
    sleep(0.5)
 
    # 겟잇뷰티 정보 있는지
    getbu_info = driver.find_elements_by_css_selector('.getbu_list ul li a')
    
    rows = []
    while(True):
        try:  
            item_info = driver.find_element_by_css_selector('.prd_info_box') 
            price = driver.find_element_by_css_selector('.prd_info_box .price').text
            getbu_info = driver.find_elements_by_css_selector('.getbu_list ul li a') 
            fullname = driver.find_element_by_css_selector('.prd_view dl dt').text
            
            # 제품설명
            t = item_info.text.split('\n')
            t = t[len(t)-1].strip()
  
            # 겟잇뷰티 정보 있는지 확인
            link = ''
            g_info = ''
            if len(getbu_info) > 0:
                getbu_info = getbu_info[0]
                link = getbu_info.get_attribute('href')
                g_info = getbu_info.text
                
            rows.append([item, price, t, fullname, link, g_info])
            
            break
        except Exception as e:
            #print(e)
            print(item, t, w, p)
            del rows[:]
            rows = [] 
            sleep(1)  
            item_info = driver.find_element_by_css_selector('.prd_info_box') 
            price = driver.find_element_by_css_selector('.prd_info_box .price').text
            getbu_info = driver.find_elements_by_css_selector('.getbu_list ul li a')
            fullname = driver.find_element_by_css_selector('.prd_view dl dt').text
    return rows

def get_item_detail_group(items): 
    driver =  webdriver.PhantomJS()
    for item in items: 
        item_detail = get_item_detail(item, driver)
        all_items_detail.extend(item_detail)
    driver.close() 

In [4]:
driver =  webdriver.PhantomJS()

In [7]:
get_item_detail(30551, driver)

[[30551,
  '9g / 12,000',
  '컬러의 선명한 발색, 광택 및 지속력을 높여주는 모디 탑 젤',
  '아리따움 모디 탑 젤',
  '',
  '']]

In [11]:
item_df = pd.read_csv('data/item.csv')

In [72]:
!killall phantomjs

No matching processes belonging to you were found


In [34]:
all_items_detail= []
for cnt in range(1, 11): 
    start = 518 * (cnt-1)
    end = 518 * cnt-1
    items = item_df.loc[start:end]['id_item'].values
    th = threading.Thread(target=get_item_detail_group, args=(items,))
    th.start()

In [66]:
len(all_items_detail)

5180

In [67]:
detail_df = pd.DataFrame(all_items_detail,columns=['id_item','price','info','fullname','gb_link','gb_info'])
detail_df

,id_item,price,info,fullname,gb_link,gb_info
0,27561,"1.8g / 17,000",풍부한 컬러와 광택과 미끄러질 듯 부드러운 젤 텍스처를 지닌 아이라이너로 하루 종일...,에스쁘아 브론즈 페인팅 워터프루프 아이펜슬 (브룸스트리트),http://getitbeauty.m.tving.com/BeautyTip/View?...,[2012.11.14] 41화 Ⅲ. 찰떡 메이크업
1,32491,"18,000",또렷한 윤곽 메이크업을 위한 납작하고 넓은 브러시,리얼테크닉 스컬프팅브러시,,
2,28834,"4.9 * 2ea / 6,500","스타일리쉬한 네일 코디가 가능한 네일 듀오로, 샵에서 받던 화려한 네일 아트를 누구...",브이디엘 페스티벌 네일 듀오,,
3,26834,"300ml / 49,000","웨이브 강화,수분공급,자연스러운 고정력,윤기 강화,잔여물이 남지 않음",모로칸오일 인텐스 컬 크림,http://getitbeauty.m.tving.com/BeautyTip/View?...,[2011.09.14] 33화 Ⅲ. 셀프앞머리 연출(1)
4,27647,"2.8g / 28,000원",파우더 타입의 펄 아이섀도우로 아주 자연스럽게 촉촉해 보이는 연출부터 화려하고 환상...,메이크업포에버 스타 파우더 #975,http://getitbeauty.m.tving.com/BeautyTip/View?...,[2012.09.12] 32화 Ⅲ. 제형+제형을 활용한 메이크업
5,32607,50ml / 18000,"백탁현상, 유분감을 꽉 잡아 산뜻한 마무리로 자외선으로부터 피부를 보호하는, 성공적...",문샷 포맨 하이 컷 선크림,http://getitbeauty.m.tving.com/BeautyTip/View?...,[2017.08.09] 22화 Ⅰ. 최.초.공.개 화장대 속 리얼 뷰티템은?
6,27560,"1.8g / 17,000",풍부한 컬러와 광택 + 미끄러질 듯 부드러운 젤 텍스처 + 하루 종일 지속되는 롱웨...,에스쁘아 브론즈 페인팅 워터프루프 아이펜슬 (탠),http://getitbeauty.m.tving.com/BeautyTip/View?...,[2012.08.01] 26화 Ⅳ. 작은 얼굴을 만드는 부위별 섀딩법
7,26833,"300ml / 49,000원",-부드러운 고정력<br/>-선명도와 윤기강화<br/>-푸석거림 제거<br/>-수분 공급,모로칸오일 하이드레이팅 스타일링 크림,,
8,19290,"5g / 14,000",자연스러운 발색력을 지닌 제품입니다.,바닐라코 투 아이즈 듀오 스퀘어드,http://getitbeauty.m.tving.com/VOD/SELFView/35769,겟잇뷰티 SELF < 골든 카키 스모키 메이크업 >
9,27111,"10g / 65,000",화이트닝 성분인 M-트라넥사믹 애씨드 미세입자로 가공하여 마무리가 매끄럽습니다. 보...,시세이도 화이트 루센트 브라이트닝 스팟 컨트롤 파운데이션,http://getitbeauty.m.tving.com/BeautyTip/View?...,[2011.04.06] 10화 Ⅲ. 센스있는 들러리 메이크업 - 베이스


In [68]:
!ls data

item.csv        item_detail.csv


In [69]:
detail_df.to_csv('data/item_detail.csv',index=False)

In [70]:
pd.read_csv('data/item_detail.csv')

,id_item,price,info,fullname,gb_link,gb_info
0,27561,"1.8g / 17,000",풍부한 컬러와 광택과 미끄러질 듯 부드러운 젤 텍스처를 지닌 아이라이너로 하루 종일...,에스쁘아 브론즈 페인팅 워터프루프 아이펜슬 (브룸스트리트),http://getitbeauty.m.tving.com/BeautyTip/View?...,[2012.11.14] 41화 Ⅲ. 찰떡 메이크업
1,32491,"18,000",또렷한 윤곽 메이크업을 위한 납작하고 넓은 브러시,리얼테크닉 스컬프팅브러시,NaN,NaN
2,28834,"4.9 * 2ea / 6,500","스타일리쉬한 네일 코디가 가능한 네일 듀오로, 샵에서 받던 화려한 네일 아트를 누구...",브이디엘 페스티벌 네일 듀오,NaN,NaN
3,26834,"300ml / 49,000","웨이브 강화,수분공급,자연스러운 고정력,윤기 강화,잔여물이 남지 않음",모로칸오일 인텐스 컬 크림,http://getitbeauty.m.tving.com/BeautyTip/View?...,[2011.09.14] 33화 Ⅲ. 셀프앞머리 연출(1)
4,27647,"2.8g / 28,000원",파우더 타입의 펄 아이섀도우로 아주 자연스럽게 촉촉해 보이는 연출부터 화려하고 환상...,메이크업포에버 스타 파우더 #975,http://getitbeauty.m.tving.com/BeautyTip/View?...,[2012.09.12] 32화 Ⅲ. 제형+제형을 활용한 메이크업
5,32607,50ml / 18000,"백탁현상, 유분감을 꽉 잡아 산뜻한 마무리로 자외선으로부터 피부를 보호하는, 성공적...",문샷 포맨 하이 컷 선크림,http://getitbeauty.m.tving.com/BeautyTip/View?...,[2017.08.09] 22화 Ⅰ. 최.초.공.개 화장대 속 리얼 뷰티템은?
6,27560,"1.8g / 17,000",풍부한 컬러와 광택 + 미끄러질 듯 부드러운 젤 텍스처 + 하루 종일 지속되는 롱웨...,에스쁘아 브론즈 페인팅 워터프루프 아이펜슬 (탠),http://getitbeauty.m.tving.com/BeautyTip/View?...,[2012.08.01] 26화 Ⅳ. 작은 얼굴을 만드는 부위별 섀딩법
7,26833,"300ml / 49,000원",-부드러운 고정력<br/>-선명도와 윤기강화<br/>-푸석거림 제거<br/>-수분 공급,모로칸오일 하이드레이팅 스타일링 크림,NaN,NaN
8,19290,"5g / 14,000",자연스러운 발색력을 지닌 제품입니다.,바닐라코 투 아이즈 듀오 스퀘어드,http://getitbeauty.m.tving.com/VOD/SELFView/35769,겟잇뷰티 SELF < 골든 카키 스모키 메이크업 >
9,27111,"10g / 65,000",화이트닝 성분인 M-트라넥사믹 애씨드 미세입자로 가공하여 마무리가 매끄럽습니다. 보...,시세이도 화이트 루센트 브라이트닝 스팟 컨트롤 파운데이션,http://getitbeauty.m.tving.com/BeautyTip/View?...,[2011.04.06] 10화 Ⅲ. 센스있는 들러리 메이크업 - 베이스
